In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client crewai

In [2]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

In [3]:
def fetch_unread_emails(service):
    results = service.users().messages().list(userId='me', labelIds=['INBOX', 'UNREAD']).execute()
    messages = results.get('messages', [])
    return messages

In [4]:
from crewai import Agent, Task

def process_email_with_crewai(email_content):
    # Define CrewAI agent
    responder_agent = Agent(
        role="Email Responder",
        goal="Generate a polite and professional response to the email.",
        backstory="You are an AI assistant trained to handle email communications."
    )

    # Define the task
    response_task = Task(
        description=f"Respond to the following email: {email_content}",
        agent=responder_agent
    )

    # Execute the task
    response = response_task.execute()
    return response


In [5]:
def send_response(service, message_id, response):
    email_body = {
        'raw': base64.urlsafe_b64encode(
            f"From: me\nTo: {sender_email}\nSubject: Re: {subject}\n\n{response}".encode('utf-8')
        ).decode('utf-8')
    }
    service.users().messages().send(userId='me', body=email_body).execute()
    print(f"Response sent to {sender_email}")

In [6]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Make sure 'credentials.json' is in the same directory as your script
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

In [7]:
!python main.py

python3: can't open file '/content/main.py': [Errno 2] No such file or directory
